In [2]:
!pip install uv
!uv pip install -U transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 123.1 MB/s eta 0:00:00
Using Python 3.11.13 environment at: /usr
Resolved 36 packages in 587ms
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
⠙ Preparing packages... (0/11)
packaging            ------------------------------ 64.00 KiB/64.91 KiB
⠙ Preparing packages... (0/11)
packaging            ------------------------------ 64.00 KiB/64.91 KiB
⠙ Preparing packages... (0/11)
packaging            ------------------------------ 64.00 KiB/64.91 KiB
⠙ Preparing packages... (0/11)
packaging            ------------------------------ 64.00 KiB/64.91 KiB
frozenlist           ------------------------------     0 B/230.80 KiB
⠙ Preparing packages... (0/11)
packaging            ------------------------------ 64.00 KiB/64.91 KiB
frozenlist           ------------------------------     0 B/230.80 KiB
⠙ Pre

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceM4/VQAv2", split="validation")

In [ ]:
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

correct_blip = 0
total = 0

for item in tqdm(dataset, desc="BLIP VQA"):
    image: Image.Image = item["image"]
    question: str = item["question"]
    true_answer: str = item["multiple_choice_answer"]

    inputs = processor(images=image, text=question, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs)
    pred = processor.decode(generated_ids[0], skip_special_tokens=True)

    if pred.strip().lower() == true_answer.strip().lower():
        correct_blip += 1
    total += 1

print(f"BLIP Accuracy: {correct_blip/total:.4f}")


In [ ]:
from transformers import pipeline

pipe_llava = pipeline(
    "visual-question-answering",
    model="llava-hf/llava-1.5-7b-hf",
    device=0 if torch.cuda.is_available() else -1
)

correct_llava = 0
total = 0

for item in tqdm(dataset, desc="LLaVA VQA"):
    image: Image.Image = item["image"]
    question: str = item["question"]
    true_answer: str = item["multiple_choice_answer"]

    out = pipe_llava({"image": image, "question": question})
    pred = out["answer"]

    if pred.strip().lower() == true_answer.strip().lower():
        correct_llava += 1
    total += 1

print(f"LLaVA-1.5 Accuracy: {correct_llava/total:.4f}")
